In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

C:\Users\user\.conda\envs\panadas-tutorial\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df=pd.read_csv('bear.csv')
df.head()

,Month,Monthly beer production
0,1956-01,93.2
1,1956-02,96.0
2,1956-03,95.2
3,1956-04,77.1
4,1956-05,70.9


In [3]:
df.rename(columns = {'Monthly beer production':'production'}, inplace = True)

In [4]:
df = df.set_index('Month')

In [5]:
df.isnull().sum()

production    0
dtype: int64

In [6]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(df['production'])

# print the test results
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t{}: {}'.format(key, value))
    
# check if the time series is stationary
if result[1] < 0.05 and result[0] < result[4]['5%']:
    print('The time series is stationary.')
else:
    print('The time series is not stationary.')

ADF Statistic: -2.282661418787579
p-value: 0.17762099829132444
Critical Values:
	1%: -3.4447087976702284
	5%: -2.867871300049488
	10%: -2.5701423432047443
The time series is not stationary.


In [7]:
# perform first-order differencing to make it stationary
df['diff_1'] = df['production'].diff()

df = df.fillna(df['diff_1'].mean())
# check the stationarity of the differenced time series
from statsmodels.tsa.stattools import adfuller
result = adfuller(df['diff_1'].dropna())
if result[1] < 0.05 and result[0] < result[4]['5%']:
    print('The time series is stationary after first-order differencing.')
else:
    print('The time series is not stationary after first-order differencing.')

The time series is stationary after first-order differencing.


In [1]:
df

NameError: name 'df' is not defined

In [9]:
new_df=pd.DataFrame()
new_df=df

In [10]:
new_df['Retrive']=new_df['production'].copy()

In [11]:
new_df

,production,diff_1,Retrive
Month,,,
1956-01,93.2,0.125895,93.2
1956-02,96.0,2.800000,96.0
1956-03,95.2,-0.800000,95.2
1956-04,77.1,-18.100000,77.1
1956-05,70.9,-6.200000,70.9
...,...,...,...
1995-04,127.0,-25.000000,127.0
1995-05,151.0,24.000000,151.0
1995-06,130.0,-21.000000,130.0


In [12]:
new_df = new_df.reset_index(drop=False)

In [13]:
new_df.to_csv('Stationary.csv', index=False)

In [14]:
df_diff={}
df_diff=pd.DataFrame(df_diff)
df_diff['diff_1']=df['diff_1']
df_diff

,diff_1
Month,
1956-01,0.125895
1956-02,2.800000
1956-03,-0.800000
1956-04,-18.100000
1956-05,-6.200000
...,...
1995-04,-25.000000
1995-05,24.000000
1995-06,-21.000000


In [15]:
# Split the data into training and testing sets
train_size = int(len(df_diff) * 0.8)
train_data, test_data = df_diff[:train_size], df_diff[train_size:]

In [16]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [17]:
# Perform ARIMA modeling
arima_model = ARIMA(train_data['diff_1'], order=(2, 1, 0)).fit()
arima_pred = arima_model.forecast(len(test_data))
arima_mae = mean_absolute_error(test_data['diff_1'], arima_pred)
arima_rmse = np.sqrt(mean_squared_error(test_data['diff_1'], arima_pred))
print('ARIMA MAE: {:.2f}'.format(arima_mae))
print('ARIMA RMSE: {:.2f}'.format(arima_rmse))

C:\Users\user\.conda\envs\panadas-tutorial\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\user\.conda\envs\panadas-tutorial\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\user\.conda\envs\panadas-tutorial\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


ARIMA MAE: 17.23
ARIMA RMSE: 22.37


In [18]:
#now predict with baseline

In [19]:
BaseLine=df_diff['diff_1'].mean()

In [20]:
# Create a DataFrame for predictions with a datetime index
predictions = pd.DataFrame(index=df.index)

# Set the predicted values to the mean value
predictions['baseline'] = BaseLine
predictions=predictions.iloc[:96, :]
# Calculate the MAE and RMSE
mae = mean_absolute_error(test_data['diff_1'], predictions['baseline'])
rmse = mean_squared_error(test_data['diff_1'], predictions['baseline'], squared=False)

# Print the MAE and RMSE
print('MAE:', mae)
print('RMSE:', rmse)

MAE: 16.952083333333334
RMSE: 22.109622566002354


In [78]:
#as here we can observe that baseline mae and rmse values are nearest
#now we try to make another model with more precision with CNN